# 02 Nettoyage allocations

# Objectifs
1. Constituer un jeu de données comprenant les différentes allocations par mois et par département

In [28]:
import pandas as pd
import datetime

In [63]:
path = "../data_raw/Données_mensuelles_prestations_solidarité_Mai22_VF.xlsx"


In [30]:
df1 = pd.read_excel(
    path,
    sheet_name="Tableau 3",
    skiprows=5,
    nrows=1
)

df1.head()

df1.columns

Index(['Département', 'Unnamed: 1', 'Ain', 'Unnamed: 3', 'Aisne', 'Unnamed: 5',
       'Allier', 'Unnamed: 7', 'Alpes-de-Haute-Provence', 'Unnamed: 9',
       ...
       'Guadeloupe**', 'Unnamed: 195', 'Martinique', 'Unnamed: 197', 'Guyane',
       'Unnamed: 199', 'La Réunion', 'Unnamed: 201', 'Mayotte',
       'Unnamed: 203'],
      dtype='object', length=204)

In [31]:
df = pd.read_excel(
    path, 
    sheet_name="Tableau 3",
    skiprows=7, 
    names=df1.columns)

df = df.rename(columns={"Département": "month"})

df = df[[c for c in df.columns if "Unnamed:" not in c]]

df.head()


,month,Ain,Aisne,Allier,Alpes-de-Haute-Provence,Hautes-Alpes,Alpes-Maritimes,Ardèche,Ardennes,Ariège,...,Essonne,Hauts-de-Seine,Seine-Saint-Denis,Val-de-Marne,Val-d'Oise,Guadeloupe**,Martinique,Guyane,La Réunion,Mayotte
0,2017-02-01 00:00:00,7630.0,17510.0,9490.0,4300.0,2860.0,24930.0,6600.0,10770.0,5970.0,...,26050.0,30230.0,82590.0,40540.0,32470.0,44730.0,36100.0,22380.0,94760.0,6380.0
1,2017-03-01 00:00:00,7630.0,17620.0,9530.0,4330.0,2860.0,25160.0,6610.0,10810.0,6030.0,...,25940.0,30390.0,82680.0,40420.0,32490.0,44630.0,36020.0,22230.0,94710.0,6330.0
2,2017-04-01 00:00:00,7630.0,17650.0,9590.0,4360.0,2900.0,25080.0,6590.0,10800.0,6070.0,...,25880.0,30430.0,82700.0,40360.0,32530.0,44550.0,35880.0,22080.0,94770.0,6210.0
3,2017-05-01 00:00:00,7590.0,17620.0,9570.0,4350.0,2910.0,24920.0,6560.0,10770.0,6090.0,...,25970.0,30480.0,82800.0,40500.0,32590.0,44720.0,35850.0,22310.0,94690.0,6190.0
4,2017-06-01 00:00:00,7600.0,17500.0,9580.0,4330.0,2890.0,24720.0,6560.0,10670.0,6050.0,...,25990.0,30430.0,82540.0,40560.0,32550.0,44750.0,35790.0,22510.0,94650.0,6130.0


In [32]:
df = df[df['month'].apply(type) == datetime.datetime]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 0 to 61
Columns: 102 entries, month to Mayotte
dtypes: float64(101), object(1)
memory usage: 49.9+ KB


In [33]:
df = df.melt(
    id_vars=["month"],
    value_vars=df.columns[1:],
    value_name="nb_rsa",
)

df = df.rename(columns={"variable": "departement"})

In [34]:
df.head()

,month,departement,nb_rsa
0,2017-02-01,Ain,7630.0
1,2017-03-01,Ain,7630.0
2,2017-04-01,Ain,7630.0
3,2017-05-01,Ain,7590.0
4,2017-06-01,Ain,7600.0


In [35]:
df["departement"] = df["departement"].str.replace("*", "", regex=False)

In [42]:
df["departement"].value_counts()

Ain                     62
Pyrénées-Atlantiques    62
Haute-Savoie            62
Savoie                  62
Sarthe                  62
                        ..
Haute-Garonne           62
Gard                    62
Finistère               62
Eure-et-Loir            62
Mayotte                 62
Name: departement, Length: 101, dtype: int64

In [82]:
dfs = []

names = [
    "nb_rsa",
    "nb_aah",
    "nb_ass",
    "nb_prime_activite",
    "nb_aide_logement",
    "nb_garantie_jeune",
    "nb_garantie_jeune_mois"
]

for i, name in zip(range(3, 10), names):
    df1 = pd.read_excel(
        path,
        sheet_name=f"Tableau {i}",
        skiprows=5,
        nrows=1
    )

    df1.columns

    df = pd.read_excel(
        path, 
        sheet_name=f"Tableau {i}",
        skiprows=7, 
        names=df1.columns)

    df = df.rename(columns={"Département": "month"})

    df = df[[c for c in df.columns if "Unnamed:" not in c]]

    df = df[df['month'].apply(type) == datetime.datetime]
    df.info()

    df = df.melt(
        id_vars=["month"],
        value_vars=df.columns[1:],
        value_name=name,
    )

    df = df.rename(columns={"variable": "departement"})
    df["departement"] = df["departement"].str.replace("-", " ", regex=False).str.lower().str.strip()

    df["departement"] = df["departement"].str.replace("*", "", regex=False)
    dfs.append(df.copy())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 0 to 61
Columns: 102 entries, month to Mayotte
dtypes: float64(101), object(1)
memory usage: 49.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 0 to 61
Columns: 102 entries, month to Mayotte
dtypes: float64(101), object(1)
memory usage: 49.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 61 entries, 0 to 60
Columns: 102 entries, month to Mayotte
dtypes: float64(101), object(1)
memory usage: 49.1+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 0 to 61
Columns: 102 entries, month to Mayotte
dtypes: float64(101), object(1)
memory usage: 49.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 62 entries, 0 to 61
Columns: 102 entries, month to Mayotte
dtypes: float64(101), object(1)
memory usage: 49.9+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 61 entries, 0 to 60
Columns: 102 entries, month to Mayotte
dtypes: float64(101), object(1)
memory usage: 49.1+ KB
<class 'pandas.c

In [83]:
df = dfs[0]

for df_temp in dfs[1:]:
    df = pd.merge(
        left=df,
        right=df_temp,
        how="outer",
        on=["month", "departement"]
    )

In [84]:
df.head()

,month,departement,nb_rsa,nb_aah,nb_ass,nb_prime_activite,nb_aide_logement,nb_garantie_jeune,nb_garantie_jeune_mois
0,2017-02-01,ain,7630.0,6690.0,2270.0,18150.0,37660.0,183.0,163.0
1,2017-03-01,ain,7630.0,6700.0,2260.0,18490.0,37920.0,219.0,191.0
2,2017-04-01,ain,7630.0,6730.0,2250.0,18510.0,38070.0,211.0,181.0
3,2017-05-01,ain,7590.0,6770.0,2210.0,18820.0,38160.0,245.0,195.0
4,2017-06-01,ain,7600.0,6800.0,2180.0,18940.0,38170.0,275.0,235.0


In [85]:
df.isna().sum()

month                       0
departement                 0
nb_rsa                      0
nb_aah                      0
nb_ass                    101
nb_prime_activite           0
nb_aide_logement            0
nb_garantie_jeune         101
nb_garantie_jeune_mois    303
dtype: int64

In [88]:
df[df["nb_ass"].isna()]

,month,departement,nb_rsa,nb_aah,nb_ass,nb_prime_activite,nb_aide_logement,nb_garantie_jeune,nb_garantie_jeune_mois
61,2022-03-01,ain,7700.0,7180.0,NaN,34860.0,33380.0,NaN,NaN
123,2022-03-01,aisne,17800.0,11270.0,NaN,39190.0,45720.0,NaN,NaN
185,2022-03-01,allier,10090.0,8100.0,NaN,24070.0,28800.0,NaN,NaN
247,2022-03-01,alpes de haute provence,4400.0,3420.0,NaN,12270.0,13000.0,NaN,NaN
309,2022-03-01,hautes alpes,3100.0,2890.0,NaN,10450.0,10040.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6013,2022-03-01,guadeloupe,43270.0,10360.0,NaN,30030.0,41710.0,NaN,NaN
6075,2022-03-01,martinique,34470.0,9200.0,NaN,31880.0,36020.0,NaN,NaN
6137,2022-03-01,guyane,23120.0,2950.0,NaN,12650.0,18720.0,NaN,NaN
6199,2022-03-01,la réunion,96410.0,21410.0,NaN,86620.0,115110.0,NaN,NaN


In [78]:
print(len(df["departement"].unique()))
df["departement"].unique()

102


array(['ain', 'aisne', 'allier', 'alpes de haute provence',
       'hautes alpes', 'alpes maritimes', 'ardèche', 'ardennes', 'ariège',
       'aube', 'aude', 'aveyron', 'bouches du rhône', 'calvados',
       'cantal', 'charente', 'charente maritime', 'cher', 'corrèze',
       'corse du sud', 'haute corse', "côte d'or", "côtes d'armor",
       'creuse', 'dordogne', 'doubs', 'drôme', 'eure', 'eure et loir',
       'finistère', 'gard', 'haute garonne', 'gers', 'gironde', 'hérault',
       'ille et vilaine', 'indre', 'indre et loire', 'isère', 'jura',
       'landes', 'loir et cher', 'loire', 'haute loire',
       'loire atlantique', 'loiret', 'lot', 'lot et garonne', 'lozère',
       'maine et loire', 'manche', 'marne ', 'haute marne', 'mayenne',
       'meurthe et moselle', 'meuse', 'morbihan', 'moselle', 'nièvre',
       'nord', 'oise', 'orne', 'pas de calais', 'puy de dôme',
       'pyrénées atlantiques', 'hautes pyrénées', 'pyrénées orientales',
       'bas rhin', 'haut rhin', 'rhône'

In [89]:
print(pd.Series(df["departement"].unique()).apply(lambda x: "elfort" in x).sum())
pd.Series(df["departement"].unique())[pd.Series(df["departement"].unique()).apply(lambda x: "elfort" in x)]

1


90    territoire de belfort
dtype: object

In [90]:
print(pd.Series(df["departement"].unique()).apply(lambda x: "'" in x).sum())
pd.Series(df["departement"].unique())[pd.Series(df["departement"].unique()).apply(lambda x: "'" in x)]

3


21        côte d'or
22    côtes d'armor
95       val d'oise
dtype: object

In [91]:
print(pd.Series(df["departement"].unique()).apply(lambda x: "rn" in x).sum())
pd.Series(df["departement"].unique())[pd.Series(df["departement"].unique()).apply(lambda x: "rn" in x)]

7


51              marne
52        haute marne
61               orne
77     seine et marne
81               tarn
82    tarn et garonne
94       val de marne
dtype: object

Le reste des données manquantes vient sans doute d'un manque de chiffre pour:
- 1 mois pour `nb_ass` et `nb_garantie_jeune`
- 3 mois pour `nb_garantie_jeune_mois`

## 2. Sauvegarde

In [92]:
df.to_csv("../data_clean/clean_allocations.csv", index=False)

In [93]:
df = pd.read_csv("../data_clean/clean_allocations.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6262 entries, 0 to 6261
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   month                   6262 non-null   object 
 1   departement             6262 non-null   object 
 2   nb_rsa                  6262 non-null   float64
 3   nb_aah                  6262 non-null   float64
 4   nb_ass                  6161 non-null   float64
 5   nb_prime_activite       6262 non-null   float64
 6   nb_aide_logement        6262 non-null   float64
 7   nb_garantie_jeune       6161 non-null   float64
 8   nb_garantie_jeune_mois  5959 non-null   float64
dtypes: float64(7), object(2)
memory usage: 440.4+ KB


In [94]:
df.head()

,month,departement,nb_rsa,nb_aah,nb_ass,nb_prime_activite,nb_aide_logement,nb_garantie_jeune,nb_garantie_jeune_mois
0,2017-02-01,ain,7630.0,6690.0,2270.0,18150.0,37660.0,183.0,163.0
1,2017-03-01,ain,7630.0,6700.0,2260.0,18490.0,37920.0,219.0,191.0
2,2017-04-01,ain,7630.0,6730.0,2250.0,18510.0,38070.0,211.0,181.0
3,2017-05-01,ain,7590.0,6770.0,2210.0,18820.0,38160.0,245.0,195.0
4,2017-06-01,ain,7600.0,6800.0,2180.0,18940.0,38170.0,275.0,235.0


In [95]:
df["month"].min(), df["month"].max()

('2017-02-01', '2022-03-01')